In [1]:
import numpy as np
import utils
import matplotlib.pyplot as plt
from lava.proc.lif.process import LIF
from lava.proc.io.source import RingBuffer
from lava.proc.monitor.process import Monitor
from lava.magma.core.run_conditions import RunSteps
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.proc.dense.process import Dense
from lava.lib.dnf.kernels.kernels import MultiPeakKernel
from lava.lib.dnf.operations.operations import Convolution
from lava.lib.dnf.connect.connect import connect
from lava.lib.dnf.operations.operations import ExpandDims
from lava.lib.dnf.operations.operations import Weights

In [11]:
input_px = LIF(shape=(2, ))

In [12]:
timesteps = 3
vel_input = np.zeros((2, timesteps))
vel_input[0, 0] = 100
vel_input[1, -1] = 100
#input neurons
vel_input_n = RingBuffer(vel_input)

In [13]:
#vel neurons
speed_low = LIF(shape=(2, ), du=0.1)
speed_mid = LIF(shape=(2, ), du=0.3)
speed_high = LIF(shape=(2, ), du=0.5)

vel_input_n.s_out.connect(speed_low.a_in)
vel_input_n.s_out.connect(speed_mid.a_in)
vel_input_n.s_out.connect(speed_high.a_in)

In [14]:
monitor_events = Monitor()
monitor_events.probe(vel_input_n.s_out, timesteps)

In [15]:
# monitor_low = Monitor()
# monitor_mid = Monitor()
# monitor_high = Monitor()
# monitor_low.probe(speed_low.u, timesteps)
# monitor_mid.probe(speed_mid.u, timesteps)
# monitor_high.probe(speed_high.u, timesteps)

In [16]:
s_monitor_low = Monitor()
s_monitor_mid = Monitor()
s_monitor_high = Monitor()
s_monitor_low.probe(speed_low.s_out, timesteps)
s_monitor_mid.probe(speed_mid.s_out, timesteps)
s_monitor_high.probe(speed_high.s_out, timesteps)

In [17]:
v_monitor_low = Monitor()
v_monitor_mid = Monitor()
v_monitor_high = Monitor()
v_monitor_low.probe(speed_low.v, timesteps)
v_monitor_mid.probe(speed_mid.v, timesteps)
v_monitor_high.probe(speed_high.v, timesteps)

In [18]:
run_condition = RunSteps(num_steps=timesteps, blocking=True)
vel_input_n.run(condition=run_condition, run_cfg=Loihi1SimCfg())

[Loihi1SimCfg]: Using the first PyProcessModel PySendModelFloat available for Process Process_13::RingBuffer.
[Loihi1SimCfg]: Using the first PyProcessModel PyLifModelFloat available for Process Process_15::LIF.
[Loihi1SimCfg]: Using the first PyProcessModel PyLifModelFloat available for Process Process_14::LIF.
[Loihi1SimCfg]: Using the first PyProcessModel PyLifModelFloat available for Process Process_16::LIF.


In [91]:
monitor_events.get_data()

{'Process_53': {'s_out': array([[100.,   0.],
         [  0.,   0.],
         [  0., 100.]])}}

In [92]:
s_monitor_low.get_data()

{'Process_54': {'s_out': array([[1., 0.],
         [1., 0.],
         [1., 1.]])}}

In [93]:
monitor_mid.get_data()

NameError: name 'monitor_mid' is not defined

In [94]:
monitor_high.get_data()

NameError: name 'monitor_high' is not defined

In [95]:
v_monitor_low.get_data()

{'Process_54': {'v': array([[0., 0.],
         [0., 0.],
         [0., 0.]])}}

In [10]:
vel_input_n.stop()